In [1]:
import os
import numpy as np
import pandas as pd
import random as rn

import tensorflow as tf
import keras

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Sequential

from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Activation, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import regularizers

2025-05-05 12:43:42.515569: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746449022.804383      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746449022.876631      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
dir_path = "/kaggle/input/rvlcdip/"
os.listdir(dir_path)

['labels_final.csv', 'data_final']

In [3]:
train_df = pd.read_csv('/kaggle/input/rvlcdip/labels_final.csv')
train_df.head()

,path,label
0,imagesv/v/o/h/voh71d00/509132755+-2755.tif,3
1,imagesl/l/x/t/lxt19d00/502213303.tif,3
2,imagesx/x/e/d/xed05a00/2075325674.tif,2
3,imageso/o/j/b/ojb60d00/517511301+-1301.tif,3
4,imagesq/q/z/k/qzk17e00/2031320195.tif,7


In [6]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2,
    dtype=None
)

In [8]:
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="/kaggle/input/rvlcdip/data_final/",
    x_col="path",
    y_col="label",
    subset="training",
    batch_size=100,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(224, 224)
)

Found 38400 validated image filenames.


In [10]:


valid_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="/kaggle/input/rvlcdip/data_final/",
    x_col="path",
    y_col="label",
    subset="validation",
    batch_size=100,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(224, 224)
)


Found 9600 validated image filenames.


In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), input_shape=(224, 224, 3), activation = 'relu'),
    MaxPool2D(pool_size=(2, 2)),

    Conv2D(64, (3,3), input_shape=(224, 224), activation = 'relu'),
    MaxPool2D(pool_size=(2, 2)),

    Conv2D(128, (3,3), input_shape=(224, 224), activation = 'relu'),
    MaxPool2D(pool_size=(2, 2)),

    Flatten(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │       5,537,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 16)                  │             528 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,633,712 (21.49 MB)

 Trainable params: 5,633,712 (21.49 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
import os
import datetime

In [26]:
checkpoint_path = "kaggle/working/model_checkpoints/weights.{epoch:02d}-{val_loss:2f}.keras"
log_dir = "/kaggle/working/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
board = TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks = [model_checkpoint, reduce_lr, early_stop, board]


In [27]:
model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history = model.fit(
                    train_generator,
                    validation_data = valid_generator,
                    epochs = 50,
                    callbacks = callbacks
)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1746451252.284780     103 service.cc:148] XLA service 0x7a2938008380 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746451252.287870     103 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1746451252.287894     103 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1746451252.712354     103 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1746451263.632916     103 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of th

330/384 ━━━━━━━━━━━━━━━━━━━━ 2:07 2s/step - accuracy: 0.1846 - loss: 2.5824

In [ ]:
# model.save
save the architecture
save the weights
save any other parameters

# model.save-weights
only saving the weight parameter (kernels) - keep the size low for model

# dummy architecture
